# Example of rAIse utilisation (MNIST data)

In [ ]:
import json
import os
import sys
from configparser import ConfigParser

import numpy as np
from keras import backend as K
from keras.utils import to_categorical

sys.path.append("/".join(sys.path[0].split('/')[:-1]))

## Install dependensies

In [ ]:
!pip install -U -r requirements.mnist.txt

## Preprocess data

In [ ]:
from keras.datasets.mnist import load_data


# Load data
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = load_data()

num_classes = 10

# Input image dimensions
img_rows, img_cols = 28, 28

In [ ]:
# The data, split between train and test sets
x_train, y_train = mnist_x_train[:10000], mnist_y_train[:10000]
x_test, y_test = mnist_x_test[:1500], mnist_y_test[:1500]

if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## Create initial model (optional)

In [ ]:
import random

from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D
from keras.models import Model


batch_size = 100
epochs = 5

input_layer = Input(shape=input_shape)
x = Conv2D(filters=random.choice(range(8, 129, 8)),
           kernel_size=random.choice(range(3, 11)),
           activation="relu",
           kernel_initializer="he_normal")(input_layer)
x = MaxPooling2D(pool_size=random.choice(range(2, 5)))(x)
x = Flatten()(x)
output_layer = Dense(num_classes,
                     kernel_initializer="he_normal",
                     activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.fit(x_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
# Save model
path_dir = "../InitialModel"
if not os.path.exists(path_dir):
    os.makedirs(path_dir)

model.save_weights(os.path.join(path_dir, "model_weights.h5"), overwrite=True)
with open(os.path.join(path_dir, "model.config"), "w") as f:
    json.dump(model.to_json(), f)

## rAIse integration

### Trainer object

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from examples.itrainer import ITrainer


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


class Trainer(ITrainer):
    def load_data(self):
      pass

    @classmethod
    def save_model(cls, model, path_dir="../InitialModel"):
        super().save_model(model, path_dir)

    def create_model(self):
        pass

    def train_func(self, model):
        batch_size = 100
        epochs = 5

        model.compile(loss="categorical_crossentropy",
                      optimizer="Adam",
                      metrics=["accuracy"])

        early_stop = EarlyStopping(monitor="val_loss",
                                   min_delta=0,
                                   patience=1,
                                   verbose=0,
                                   mode="auto")
        reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                                      factor=0.2,
                                      patience=1,
                                      min_lr=0.001)
        results = model.fit(x_train,
                            y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[early_stop, reduce_lr])

        train_accuracy = results.history["accuracy"][-1]
        validation_accuracy = results.history["val_accuracy"][-1]
        return train_accuracy, validation_accuracy

    def evaluate_func(self, model):
        batch_size = 200

        model.compile(loss="categorical_crossentropy",
                      optimizer="Adam",
                      metrics=["accuracy"])
        results = model.evaluate(x_test, y_test, batch_size=batch_size)
        test_accuracy = results[-1]
        return test_accuracy

### Model optimization

In [ ]:
import run_population as rp

config_file = "../project.cfg"
project_config = ConfigParser()
project_config.read(config_file)
trainer_class = Trainer

cwd = os.getcwd()
os.chdir("/".join(sys.path[0].split('/')[:-1]))

try:
    rp.run_population(project_config=project_config,
                      trainer_class=trainer_class)
except SystemExit:
    pass
finally:
    os.chdir(cwd)

### Get champion and use it

In [ ]:
from keras.models import model_from_json

from clients.api import RequestToApi


config_file = "../project.cfg"
project_config = ConfigParser()
project_config.read(config_file)
url = project_config.get("DEFAULT", "url")
project_id = project_config.get("DEFAULT", "project_id")
token = project_config.get("DEFAULT", "token")
agents_dir = project_config.get("DEFAULT", "agents_path")

# Get champion ID
api = RequestToApi(url, token)
project = api.get_project(project_id=project_id)
mutant_id = project["scoreChampionId"] if project["scoreChampionId"] else project["initialMutantId"]
champion_id = api.get_mutant(mutant_id=mutant_id)["agentId"]

# Get champion config & weights files
champion_path = os.path.join("..", agents_dir, project_id, champion_id)
champion_weights = f"{champion_path}_weights.h5"
champion_cfg = f"{champion_path}.config"

with open(champion_cfg, "r") as f:
    champion_config = json.load(f)

# Load network
model = model_from_json(champion_config)
model.load_weights(champion_weights)

In [ ]:
# New data
x_sample, y_sample = mnist_x_test[1500:2500], mnist_y_test[1500:2500]

if K.image_data_format() == "channels_first":
    x_sample = x_sample.reshape(x_sample.shape[0], 1, img_rows, img_cols)
else:
    x_sample = x_sample.reshape(x_sample.shape[0], img_rows, img_cols, 1)

x_sample = x_sample.astype("float32")
x_sample /= 255

prediction = model.predict(x_sample)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

y_pred = np.argmax(prediction, axis=1)
y_pred = y_pred.reshape(y_pred.shape[0], 1)

conf = confusion_matrix(y_sample, y_pred)
sns.heatmap(conf, annot=True, fmt="d")
plt.show()